In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import imageio
%matplotlib inline

## Download

youtube-dl --rm-cache-dir

youtube-dl -f bestvideo https://youtu.be/Fkadv0VnZkI

youtube-dl -f bestvideo https://www.youtube.com/playlist?list=PLAPUEAObdbMb747QUFsjQ2e9MPz1FkDnQ

youtube-dl -f bestvideo https://www.youtube.com/playlist?list=PLAPUEAObdbMaBtaElCDD3XD4hWO631ihN

## Preprocess

ffmpeg -i video1.webm -r 1 second_iteration/%06d_img.jpg

In [ ]:
videos_path = Path('../sport_data/')

In [ ]:
videos_suffixes = [n.suffix for n in videos_path.iterdir()]

In [ ]:
pd.Series.value_counts(videos_suffixes)

In [ ]:
command = ''
for i, p in enumerate(videos_path.iterdir()):
    if p.suffix in ['.mp4', '.webm']:
        command += f'ffmpeg -i "{p.name}" -r 10 second_iteration/vid_{i:02}_%05d.jpg; '

In [ ]:
# command

## Raw images 

In [ ]:
dataset_path = Path('../sport_data/second_iteration/')

In [ ]:
suffixes = [n.suffix for n in dataset_path.iterdir()]

In [ ]:
pd.Series.value_counts(suffixes)

In [ ]:
for p in dataset_path.iterdir():
    if p.suffix.lower() not in ['.jpg', '.png', '.jpeg']:
        p.unlink()

In [ ]:
for i in range(30):
    print(i, len(list(dataset_path.glob(f'vid_{i:02}*'))))

In [ ]:
images = []
for p in sorted(list(dataset_path.glob('vid_04*')))[540:550]:
    images.append(imageio.imread(p))

In [ ]:
import numpy as np

In [ ]:
for i in range(len(images[-5:])):
    plt.figure()
    plt.imshow(images[i])

## Run pose detection

See instructions in ./deep-high-resolution-net.pytorch/demo/run.sh

## Pose detection results

In [ ]:
bboxes = Path('/home/evgenykashin/projects/sport/deep-high-resolution-net.pytorch/demo/output2/boxes/')
poses = Path('/home/evgenykashin/projects/sport/deep-high-resolution-net.pytorch/demo/output2/poses/')

In [ ]:
for b in list(bboxes.iterdir())[:5]:
    plt.figure()
    plt.imshow(plt.imread(b))

In [ ]:
for b in list(poses.iterdir())[:5]:
    plt.figure()
    plt.imshow(plt.imread(b))

## Pose detection coords processing

In [ ]:
poses_df = pd.read_csv('/home/evgenykashin/projects/sport/deep-high-resolution-net.pytorch/demo/output2/pose-data.csv',
                       encoding = "ISO-8859-1")

In [ ]:
poses_df.box_top_left_x = poses_df.box_top_left_x.astype(int)
poses_df.box_top_left_y = poses_df.box_top_left_y.astype(int)
poses_df.box_bottom_right_x = poses_df.box_bottom_right_x.astype(int)
poses_df.box_bottom_right_y = poses_df.box_bottom_right_y.astype(int)

In [ ]:
poses_df.head()

In [ ]:
poses_df.shape

In [ ]:
import cv2
import json
from IPython.display import clear_output

In [ ]:
def extract_from_row(row):
    box_left = (row.box_top_left_x, row.box_top_left_y)
    box_right = (row.box_bottom_right_x, row.box_bottom_right_y)

    foot_left = (row.nose_x, row.nose_y)
    knee_left = (row.left_eye_x, row.left_eye_y)
    hip_left = (row.right_eye_x, row.right_eye_y)
    hip_right = (row.left_ear_x, row.left_ear_y)
    knee_right = (row.right_ear_x, row.right_ear_y)
    hip_center = (row.right_shoulder_x, row.right_shoulder_y)
    top_center = (row.left_elbow_x, row.left_elbow_y)
    wrist_left = (row.right_wrist_x, row.right_wrist_y)
    shoulder_left = (row.right_hip_x, row.right_hip_y)
    shoulder_right = (row.left_knee_x, row.left_knee_y)
    wrist_right = (row.left_ankle_x, row.left_ankle_y)
    head_top = (row.left_wrist_x, row.left_wrist_y)
    head_bottom = (row.right_elbow_x, row.right_elbow_y)
    foot_right = (row.left_shoulder_x, row.left_shoulder_y)
    
    return box_left, box_right, foot_left, knee_left, hip_left, hip_right, \
           knee_right, hip_center, top_center, wrist_left, shoulder_left, \
           shoulder_right, wrist_right, head_top, head_bottom, foot_right

In [ ]:
def draw_on_image(row, img):
    box_left, box_right, foot_left, knee_left, hip_left, hip_right, \
           knee_right, hip_center, top_center, wrist_left, shoulder_left, \
           shoulder_right, wrist_right, head_top, head_bottom, foot_right = extract_from_row(row)

    img = cv2.rectangle(img, box_left, box_right, color=(0, 255, 0), thickness=3)

    img = cv2.circle(img, foot_left, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'ft_L', foot_left, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, knee_left, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'kn_L', knee_left, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, knee_right, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'kn_R', knee_right, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, hip_left, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'hip_L', hip_left, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, hip_right, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'hip_R', hip_right, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, hip_center, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'hip', hip_center, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, top_center, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'top_C', top_center, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, wrist_left, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'wr_L', wrist_left, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, shoulder_left, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'shdr_L', shoulder_left, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, shoulder_right, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'shdr_R', shoulder_right, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, wrist_right, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'wr_R', wrist_right, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, head_top, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'head_T', head_top, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, head_bottom, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'head_B', head_bottom, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    img = cv2.circle(img, foot_right, 4, (255, 0, 0), 2)
    img = cv2.putText(img, 'ft_R', foot_right, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 255, thickness=2)
    
    return img

In [ ]:
def extract_from_image(row, img):
    box_left, box_right, foot_left, knee_left, hip_left, hip_right, \
           knee_right, hip_center, top_center, wrist_left, shoulder_left, \
           shoulder_right, wrist_right, head_top, head_bottom, foot_right = extract_from_row(row)
    box_right = (box_right[0], (knee_left[1] + knee_right[1]) // 2)
    return img[box_left[1]: box_right[1], box_left[0]: box_right[0], :]

## Extract usefull crops from images

In [ ]:
def map_coord_to_crop(coord, box_l, box_r):
    x = min(coord[0] - box_l[0], box_r[0])
    y = min(coord[1] - box_l[1], box_r[1])
    return (x, y)

def extract_from_image_with_points(row, img):
    box_left, box_right, foot_left, knee_left, hip_left, hip_right, \
           knee_right, hip_center, top_center, wrist_left, shoulder_left, \
           shoulder_right, wrist_right, head_top, head_bottom, foot_right = extract_from_row(row)
    box_right = (box_right[0], (knee_left[1] + knee_right[1]) // 2)
    
    hip_left = map_coord_to_crop(hip_left, box_left, box_right)
    hip_right = map_coord_to_crop(hip_right, box_left, box_right)
    shoulder_left = map_coord_to_crop(shoulder_left, box_left, box_right)
    shoulder_right = map_coord_to_crop(shoulder_right, box_left, box_right)
    head_bottom = map_coord_to_crop(head_bottom, box_left, box_right)
    
    return img[box_left[1]: box_right[1], box_left[0]: box_right[0], :],\
        (hip_left, hip_right, shoulder_left, shoulder_right, head_bottom)

## Resize and pad of crops

In [ ]:
def image_resize(image, keypoints, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        new_w, new_h = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        new_w, new_h = (width, int(h * r))

    # resize the image
    resized_image = cv2.resize(image.copy(), (new_w, new_h), interpolation = inter)
    
    resized_keypoints = []
    for k in keypoints:
        resized_keypoints.append((int(k[0] * (new_w / w)), int(k[1] * (new_h / h))))
    # return the resized image
    return resized_image, resized_keypoints

def image_pad(image, keypoints, width=None, height=None):
    (h, w, c) = image.shape
    if width is None and height is None:
        return image
    
    if width is None:
        raise NotImplementedError()
    else:
        pad_image = np.zeros((h, width, c), dtype=np.uint8)
        pad_keypoints = []
        
        if w < width:
            pad_size = width - w
            pad_left = pad_size // 2
            pad_right = pad_left + w
            pad_image[:, pad_left:pad_right, :] = image.copy()
            
            for k in keypoints:
                pad_keypoints.append((k[0] + pad_left, k[1]))
        else:
            crop_size = w - width
            crop_left = crop_size // 2
            crop_right = crop_left + width
            pad_image = image[:, crop_left:crop_right, :].copy()
            
            for k in keypoints:
                pad_keypoints.append((k[0] - crop_left, k[1]))
    
    return pad_image, pad_keypoints

def resize_pad(image, keypoints, width=None, height=None):
    image = image[:, :, :3]
    if image.dtype == np.float32:
        image = (image * 255).astype(np.uint8)
        
    res_image, res_keyp = image_resize(image, keypoints, height=height)
    pad_image, pad_keyp = image_pad(res_image, res_keyp, width=width)
    return pad_image, pad_keyp

### Image align

https://www.codeproject.com/Articles/865830/Point-cloud-alignment-ICP-methods-compared

https://github.com/zpincus/celltool/blob/master/celltool/numerics/image_warp.py

In [ ]:
def landmarks_match(src_im, src_landmarks, tar_landmarks): 
    """
    umeyama(src, dst, estimate_scale), 
    src/dst landmarks coord. should be (y, x)
    """
    src_size = src_im.shape
    src_tmp = [(int(xy[1]), int(xy[0])) for xy in src_landmarks]
    dst_tmp = [(int(xy[1]), int(xy[0])) for xy in tar_landmarks]
    M1 = umeyama(np.array(src_tmp), np.array(dst_tmp), True)

    result1 = cv2.warpAffine(src_im.copy(), M1.astype(np.float32)[:2], (src_size[1], src_size[0]), borderMode=cv2.BORDER_REPLICATE) 
    return result1

In [ ]:
import tqdm
from umeyama import umeyama

In [ ]:
is_debug = True

In [ ]:
small_images = 0
target_width, target_height = 512, 640
mean_keypoints = np.loadtxt('mean_keypoints.txt', dtype=np.int32) # from EDA.ipynb

for i in tqdm.tqdm(range(len(poses_df))):
    fname = poses_df.iloc[i].fname
    box_num = poses_df.iloc[i].box_num
    name = fname.split('/')[-1]

    try:
        img = plt.imread('../sport_data/second_iteration/' + name)
    except:
        print('Error with opening', name)
        continue

    row = poses_df.iloc[i]
    img, keypoints = extract_from_image_with_points(row, img)
    
    if img.shape[0] < 400:
        small_images += 1
        continue
        
    pad_image, pad_keyp = resize_pad(img, keypoints, target_width, target_height)
    aligned_img = landmarks_match(pad_image, pad_keyp, mean_keypoints)
    
    new_name = name.split('.')[0]
    if is_debug:
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.subplot(1, 2, 2)
        plt.imshow(aligned_img)
        break
    else:
        imageio.imsave(f'aligned_bodies2/{new_name}_{box_num}.jpg', aligned_img)
print('small_images', small_images)

In [ ]:
small_images